<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [3]:
# Создаем соединение с заданными параметрами
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''SELECT COUNT('id') "количество вакансий"
        FROM public.vacancies 
        '''

In [5]:
# результат запроса
pd.read_sql_query(query_3_1, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1469115235.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_1, connection)


,количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''SELECT COUNT('id') "количество работодателей"
        FROM public.employers 
        '''

In [7]:
# результат запроса
pd.read_sql_query(query_3_2, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\4057088587.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_2, connection)


,количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''SELECT COUNT('id') "количество регионов"
            FROM public.areas 
            '''

In [9]:
# результат запроса
pd.read_sql_query(query_3_3, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\4147122770.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_3, connection)


,количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''SELECT COUNT('id') "количество сфер деятельности"
            FROM public.industries 
            '''

In [11]:
# результат запроса
pd.read_sql_query(query_3_4, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2627729322.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_4, connection)


,количество сфер деятельности
0,294


***

**Выводы по предварительному анализу данных:**

Таким образом, 23501 работодателей из 1362 регионов разместили в базе данных 49197 вакансий по 294 сферам деятельности. В среднем на одного работодателя приходится 2 вакансии. Учитывая, что в России насчитывается 1117 городов, следовательно вакансии размещены в т.ч. из других стран.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [13]:
# текст запроса
query_4_1 = f'''SELECT a.name "Регион",
            COUNT(v.id) "Количество вакансий" 
            FROM vacancies v
            JOIN areas a ON v.area_id = a.id 
            GROUP BY 1
            ORDER BY 2 DESC
            '''

In [14]:
# результат запроса
pd.read_sql_query(query_4_1, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\4222419522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_1, connection)


,Регион,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


In [15]:
# построение графика к запросу 4.1
df_4_1 = pd.read_sql_query(query_4_1, connection)[:10]

fig = px.bar(
    data_frame=df_4_1, 
    x='Количество вакансий', 
    y='Регион', 
    color='Регион', 
    text='Количество вакансий', 
    height=500, 
    width=1000, 
    title='Топ-10 регионов по количеству вакансий' 
)
fig.update_layout(title_font_size=20, title_xanchor='center', title_x=0.4)

fig.write_html("plotly/img4-1.html")
fig.show()

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1130988462.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  q_4_1 = pd.read_sql_query(query_4_1, connection)[:10]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [16]:
# текст запроса
query_4_2 = f'''SELECT COUNT(id)
            FROM vacancies 
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
            '''

In [17]:
# результат запроса
pd.read_sql_query(query_4_2, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\614600247.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [18]:
# текст запроса
query_4_3 = f'''SELECT ROUND(AVG(salary_from)) "сред. значение для нижней границы зарплаты",
            ROUND(avg(salary_to)) "сред. значение для верхней границы зарплаты"
            FROM vacancies 
            WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
            '''

In [19]:
# результат запроса
pd.read_sql_query(query_4_3, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\3534961570.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,сред. значение для нижней границы,сред. значение для верхней границы
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [20]:
# текст запроса
query_4_4 = f'''SELECT schedule "тип графика",
                employment "тип трудоустройства",
            COUNT (id) "количество вакансий"
            FROM vacancies 
            GROUP BY schedule, employment
            ORDER BY 3 DESC
            '''

In [21]:
# результат запроса
pd.read_sql_query(query_4_4, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2630570843.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,тип графика,тип трудоустройства,количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [22]:
# текст запроса
query_4_5 = f'''SELECT experience "опыт работы",
            COUNT (id) "количество вакансий"
            FROM vacancies 
            GROUP BY 1
            ORDER BY 2
            '''

In [23]:
# результат запроса
pd.read_sql_query(query_4_5, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\210463167.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,опыт работы,количество вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


In [24]:
# построение графика к запросу 4.5
df_4_5 = pd.read_sql_query(query_4_5, connection)

fig = px.pie(df_4_5, values='количество вакансий', names='опыт работы', title='Соотношение вакансий в зависимости от опыта работы')
fig.update_traces(textinfo='percent+label')
fig.update_layout(title_font_size=20, title_xanchor='center', title_x=0.5)

fig.write_html("plotly/img4-5.html")
fig.show()

***

**Выводы по детальному анализу вакансий:**

* Больше всего вакансий размещены в крупных городах и столицах, таких как Москва, Санкт-Петербург, Минск, что и следовало ожидать.
* Из 49197 вакансий у 24073 заполнено хотя бы одно из двух полей с зарплатой, что составляет 49%, т.е. у половины вакнсий отсутствует уровень зарплаты.
* Среднее значение для нижней границы зарплатной вилки составляет 71065 рублей, для верхней в 1,5 раза выше - 110537 рублей. Если между ними найти среднее значение, то получится 90801 рубль.
* Для большей части вакансий (72%) требуется полная занятость и полный рабочий день. 16% работодателей предлагают удаленную работу при полной занятости.
* Требуемый опыт работы у большинства вакансий от 1 года до 3 лет (53%), на втором месте - от 3 до 6 лет (29%). 
15% работодателей готовы взять сотрудника без опыта работы, что внушает оптимизм.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
# текст запроса
query_5_1 = f'''SELECT e.name "Работодатель",
            COUNT (v.id) "Количество вакансий"
            FROM employers e
            JOIN vacancies v ON v.employer_id = e.id 
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 5
            '''

In [26]:
# результат запроса
pd.read_sql_query(query_5_1, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\3902369145.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Работодатель,Количество вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


In [27]:
# построение графика к запросу 5.1
df_5_1 = pd.read_sql_query(query_5_1, connection)

fig = px.bar(
    data_frame=df_5_1, 
    x='Работодатель', 
    y='Количество вакансий', 
    color='Работодатель', 
    text='Количество вакансий', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Топ-5 работодателей по количеству вакансий' 
)
fig.update_layout(title_font_size=20, title_xanchor='center', title_x=0.4)

fig.write_html("plotly/img5-1.html")
fig.show()

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2967476854.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [28]:
# текст запроса
query_5_2 = f'''SELECT a.name "регион",
            COUNT(e.id) "количество работодателей",
            COUNT(v.id) "количество вакансий"
            FROM areas a
            LEFT JOIN employers e ON a.id = e.area
            LEFT JOIN vacancies v ON a.id = v.area_id
            WHERE v.id IS NULL
            GROUP BY 1
            ORDER BY 2 DESC
            '''

In [29]:
# результат запроса
pd.read_sql_query(query_5_2, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2146817738.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,регион,количество работодателей,количество вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
588,Кувандык,0,0
589,Куйбышев,0,0
590,Куйбышево (Ростовская область),0,0
591,Култаево,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [30]:
# текст запроса
query_5_3 = f'''SELECT e.name "работодатель",
            COUNT(distinct v.area_id) "количество регионов"
            FROM employers e
            JOIN vacancies v ON v.employer_id = e.id 
            GROUP BY 1
            ORDER BY 2 DESC
            '''

In [31]:
# результат запроса
pd.read_sql_query(query_5_3, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\593965358.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,работодатель,количество регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [32]:
# текст запроса
query_5_4 = f'''SELECT COUNT(distinct e.id)  
            FROM employers e
            LEFT JOIN employers_industries i ON i.employer_id = e.id 
            WHERE industry_id IS NULL
            '''

In [33]:
# результат запроса
pd.read_sql_query(query_5_4, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1644714913.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [34]:
# текст запроса
query_5_5 = f'''SELECT e.name  
            FROM employers e
            JOIN employers_industries i ON i.employer_id = e.id 
            GROUP BY 1
            HAVING COUNT(distinct industry_id) = 4
            LIMIT 3
            '''

In [35]:
# результат запроса
pd.read_sql_query(query_5_5, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1169902027.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name
0,101 Интернет
1,21vek.by
2,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [36]:
# текст запроса
query_5_6 = f'''SELECT COUNT(e.employer_id)
            FROM industries i
            JOIN employers_industries e ON e.industry_id = i.id 
            WHERE i.name = 'Разработка программного обеспечения'
            '''

In [37]:
# результат запроса
pd.read_sql_query(query_5_6, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2878008418.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [38]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
My_table = page.find('table',{'class':'standard sortable'})
links = My_table.find_all('a')

town = []
for link in links[4:20]:
    if link in links[4:20]:
        town.append(link.get('title'))
    
print(tuple(town))

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [39]:
# текст запроса
query_5_7 = f'''SELECT a.name "Город-миллионник",
                COUNT(v.id) "Количество вакансий от Яндекс"
            FROM vacancies v                                                                                                        
            JOIN areas a ON a.id=v.area_id
            JOIN employers e ON e.id=v.employer_id
            WHERE e.name = 'Яндекс'
                AND a.name IN {tuple(town)}
            GROUP BY 1
        
            UNION ALL 
                                                                                             
            SELECT 'Total', 
                COUNT(v.id)
            FROM vacancies v                                                                                                        
            JOIN areas a ON a.id=v.area_id
            JOIN employers e ON e.id=v.employer_id
            WHERE e.name = 'Яндекс'
                AND a.name IN {tuple(town)}
            ORDER BY 2
            '''

In [40]:
# результат запроса
pd.read_sql_query(query_5_7, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1216511710.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Город-миллионник,Количество вакансий от Яндекс
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

**Выводы по анализу работодателей:**

* Среди работодателей больше всех вакансий у компании "Яндекс", которая опубликовала 1933 вакансий, что составляет почти 4% от всех вакансий, на втором месте - "Ростелеком" (491 вакансия).
* Также "Яндекс" лидирует по количеству регионов, в которых опубликовала свои вакансии (181 регион), на втором месте также "Ростелеком" (152 региона).  
* 17% работодателей не указали сферу деятельности в опубликованных вакансиях.
* 3553 (или 7%) работодателей в качестве сферы деятельности указали "Разработка программного обеспечения".
* Вакансии компании "Яндекс" находятся во всех городах-миллионниках России, больше всего в городе Москва. Скорей всего, большой охват территорий данной компании обусловлен удаленным форматом работы.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [41]:
# текст запроса
query_6_1 = f'''SELECT COUNT(id)
            FROM vacancies
            WHERE name ILIKE '%data%' OR name ILIKE '%данн%'
            '''

In [42]:
# результат запроса
pd.read_sql_query(query_6_1, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2603769340.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [43]:
# текст запроса
query_6_2 = f'''SELECT COUNT(id)
    FROM vacancies
    WHERE (name ILIKE '%data scientist%' 
      OR name ILIKE '%data science%'
      OR name ILIKE '%исследователь данных%'
      OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
      OR name ILIKE '%machine learning%'
      OR name ILIKE '%машинн%обучен%')
      AND (name ILIKE '%junior%' OR experience = 'Нет опыта' OR employment = 'Стажировка')
        '''

In [44]:
# результат запроса
pd.read_sql_query(query_6_2, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2428688723.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [45]:
# текст запроса
query_6_3 = f'''SELECT COUNT(id)
    FROM vacancies
    WHERE (name ILIKE '%data scientist%' 
      OR name ILIKE '%data science%'
      OR name ILIKE '%исследователь данных%'
      OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
      OR name ILIKE '%machine learning%'
      OR name ILIKE '%машинн%обучен%')
      AND (key_skills ILIKE '%SQL%' OR key_skills ILIKE '%postgres%')
         '''

In [46]:
# результат запроса
pd.read_sql_query(query_6_3, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\3902879435.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [47]:
# текст запроса
query_6_4 = f'''SELECT COUNT(id)
    FROM vacancies
    WHERE (name ILIKE '%data scientist%' 
      OR name ILIKE '%data science%'
      OR name ILIKE '%исследователь данных%'
      OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
      OR name ILIKE '%machine learning%'
      OR name ILIKE '%машинн%обучен%')
      AND key_skills ILIKE '%python%'
         '''

In [48]:
# результат запроса
pd.read_sql_query(query_6_4, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2548700122.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [49]:
# текст запроса
query_6_5 = f'''SELECT ROUND(AVG(1 + length(key_skills) - length(replace(key_skills, CHR(9), ''))), 2)
    FROM vacancies
    WHERE (name ILIKE '%data scientist%' 
      OR name ILIKE '%data science%'
      OR name ILIKE '%исследователь данных%'
      OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
      OR name ILIKE '%machine learning%'
      OR name ILIKE '%машинн%обучен%')
         '''

In [50]:
# результат запроса
pd.read_sql_query(query_6_5, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\3854041181.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [51]:
# текст запроса
query_6_6 = f'''SELECT experience "Требование к опыту",
    ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) "Средняя зарплата"
    FROM vacancies
    WHERE (name ILIKE '%data scientist%' 
      OR name ILIKE '%data science%'
      OR name ILIKE '%исследователь данных%'
      OR (name LIKE '%ML%' AND name NOT ILIKE '%HTML%')
      OR name ILIKE '%machine learning%'
      OR name ILIKE '%машинн%обучен%')
      AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    GROUP BY 1
    ORDER BY 2
         '''

In [52]:
# результат запроса
pd.read_sql_query(query_6_6, connection)

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\1796641676.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Требование к опыту,Средняя зарплата
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


In [53]:
# построение графика к запросу 6.6
df_6_6 = pd.read_sql_query(query_6_6, connection)

fig = px.bar(
    data_frame=df_6_6, 
    x='Требование к опыту', 
    y='Средняя зарплата', 
    color='Требование к опыту', 
    text='Средняя зарплата', 
    orientation='v', 
    height=500, 
    width=1000, 
    title='Средняя заработная плата в зависимости от опыта работы' 
)
fig.update_layout(title_font_size=20, title_xanchor='center', title_x=0.4)

fig.write_html("plotly/img6-6.html")
fig.show()

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\155297471.py:2: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



***

**Выводы по предметному анализу:**

* Количество вакансий, которые имеют отношение к данным (т.е. в её названии содержатся слова 'data' или 'данн') составляет 1771 или 3,6% от всех вакансий.
* Подходящих вакансий для начинающего дата-сайентиста (DS) - 51, что составляет 2,9% от вакансий, имеющих отношение к данным.
* Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres - 201, Python указан в 351 вакансии. Таким образом Python более востребован для дата-сайентиста, чем SQL.
* Количество ключевых навыков в вакансиях для DS в среднем составляет 6,4.
* Средняя зарплата, указываемая в вакансиях для DS, прямо пропорциональна требуемому опыту работы и составляет: без опыта - 74643 рубля, от 1 года до 3 лет - 139675 рублей (почти в 2 раза больше) и с опытом от 3 до 6 лет - 243115 рублей.

**Дополнительные исследования данных**

In [54]:
# найдем 10 популярных вакансий
# текст запроса
query_7_1 = f'''SELECT name "Название вакансии",
            COUNT('id') "Количество вакансий"
            FROM vacancies 
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 10
            '''
            
# построение графика к запросу 7_1
df_7_1 = pd.read_sql_query(query_7_1, connection)

fig = px.bar(
    data_frame=df_7_1, 
    x='Количество вакансий', 
    y='Название вакансии', 
    color='Название вакансии', 
    text='Количество вакансий', 
    height=500, 
    width=1000, 
    title='Топ-10 популярных вакансий' 
)
fig.update_layout(title_font_size=20, title_xanchor='center', title_x=0.4)

fig.write_html("plotly/img7-1.html")
fig.show()

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2213119005.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [55]:
# Топ-10 регионов с высокой средней зарплатой для вакансий DS
# текст запроса: 
query_7_2 = f'''SELECT a.name "Регион",
            ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) "Средняя зарплата",
            COUNT('v.id') "Количество вакансий"
            FROM vacancies v
            JOIN areas a ON v.area_id = a.id 
            WHERE (v.name ILIKE '%data scientist%' 
            OR v.name ILIKE '%data science%'
            OR v.name ILIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
            OR v.name ILIKE '%machine learning%'
            OR v.name ILIKE '%машинн%обучен%')
            AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 10
            '''
            
# результат запроса
pd.read_sql_query(query_7_2, connection)            

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\3410335308.py:21: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Регион,Средняя зарплата,Количество вакансий
0,Кипр,300000.0,1
1,Армения,268863.0,3
2,Черногория,233794.0,1
3,Турция,233794.0,1
4,Сербия,233794.0,1
5,Москва,207630.0,27
6,Белгород,200000.0,1
7,Санкт-Петербург,173335.0,7
8,Новосибирск,166333.0,3
9,Рязань,136667.0,3


In [56]:
# Топ-10 работодателей с высокой средней зарплатой для вакансий DS
# текст запроса: 
query_7_3 = f'''SELECT e.name "Работодатель",
            a.name "Регион регистрации",
            ROUND(AVG(coalesce((salary_to + salary_from)/2, salary_to, salary_from))) "Средняя зарплата",
            COUNT('v.id') "Количество вакансий"
            FROM employers e
            JOIN vacancies v ON v.employer_id = e.id 
            JOIN areas a ON e.area = a.id 
            WHERE (v.name ILIKE '%data scientist%' 
            OR v.name ILIKE '%data science%'
            OR v.name ILIKE '%исследователь данных%'
            OR (v.name LIKE '%ML%' AND v.name NOT ILIKE '%HTML%')
            OR v.name ILIKE '%machine learning%'
            OR v.name ILIKE '%машинн%обучен%')
            AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
            GROUP BY 1, 2
            ORDER BY 3 DESC
            LIMIT 10
            '''
            
# результат запроса
pd.read_sql_query(query_7_3, connection)            

C:\Users\Юзер\AppData\Local\Temp\ipykernel_11524\2944777011.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Работодатель,Регион регистрации,Средняя зарплата,Количество вакансий
0,Итон,Москва,450000.0,1
1,AppQuantum,Москва,350000.0,1
2,JSA Group,Москва,300000.0,2
3,OVISION,Санкт-Петербург,300000.0,1
4,Дром,Владивосток,300000.0,1
5,Semrush,Санкт-Петербург,300000.0,1
6,СберЛизинг,Москва,300000.0,1
7,Dbrain,Москва,275000.0,1
8,Кловер Групп,Москва,275000.0,1
9,КИБЕР-РОМ,Москва,275000.0,1


In [57]:
# закрытие соединения после окончания работы
connection.close()

# Общий вывод по проекту

* 